In [ ]:
import zipfile
import os

# 압축 파일 경로와 압축 해제 경로 설정
zip_file_path = r'C:/wockd/ultra/ultra/retina/open.zip'  # 압축 파일 경로
extract_to_path = r'C:/wockd/ultra/ultra/retina/data'     # 압축 해제 경로

# 압축 해제 경로 폴더가 없으면 생성
if not os.path.exists(extract_to_path):
    os.makedirs(extract_to_path)

# 압축 파일 해제
try:
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to_path)
    print(f"압축 해제가 완료되었습니다: {extract_to_path}")
except zipfile.BadZipFile:
    print("압축 파일이 손상되었습니다.")
except Exception as e:
    print(f"압축 해제 중 오류 발생: {e}")


압축 해제가 완료되었습니다: C:/wockd/ultra/ultra/retina/data


In [ ]:
import os
import gc
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision

from glob import glob
from tqdm.auto import tqdm
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from torchvision.transforms import v2
import albumentations as A
import cv2
from albumentations.pytorch import ToTensorV2

from torch.utils.data import Dataset, DataLoader
from transformers import Swinv2Model, ConvNextV2Model, AutoModel
import timm
from PIL import Image

torch.set_float32_matmul_precision('high')  # or 'medium' | 'high'
# os.environ['WANDB_API_KEY']='xxxxx'
# os.environ['WANDB_MODE']='online'
# os.environ['WANDB_PROJECT']='basslibrary240210'
os.environ['WANDB_MODE']='offline'

######## logger ########
import sys, logging, IPython
logger = logging.getLogger()
logging.basicConfig( handlers=[ logging.StreamHandler(stream=sys.stdout), logging.handlers.RotatingFileHandler(filename='run.log', mode='a', maxBytes=512000, backupCount=4) ] )
logging_fomatter = logging.Formatter( '%(asctime)s [%(levelname)-4.4s] %(message)s', datefmt='%m/%d %H:%M:%S' )
_ = [ h.setFormatter(logging_fomatter) for h in logger.handlers ]
logger.setLevel(logging.INFO)
def showtraceback(self, *args, **kwargs):
    logger.exception('-------Exception----------')
IPython.core.interactiveshell.InteractiveShell.showtraceback = showtraceback
logger.info('program started')

c:\Users\wockd\anaconda3\envs\newwoc\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\wockd\anaconda3\envs\newwoc\lib\site-packages\transformers\utils\generic.py:311: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(


11/20 13:06:22 [INFO] program started


In [ ]:
print(train_df['img_path'].head())  # 이미지 경로 열 이름이 'img_path'라고 가정


0    ./train/TRAIN_00000.jpg
1    ./train/TRAIN_00001.jpg
2    ./train/TRAIN_00002.jpg
3    ./train/TRAIN_00003.jpg
4    ./train/TRAIN_00004.jpg
Name: img_path, dtype: object


In [ ]:
CFG = {}
CFG['SEED'] = 42
CFG['N_SPLIT'] = 3
CFG['LABEL_SMOOTHING'] = 0.05
CFG['OPTIMIZER'] = 'AdamW'
CFG['INTERPOLATION'] = 'robidouxsharp'
CFG['PRECISION'] = '16'
# #----------------------------------
# # [9842]
#CFG['MODEL_NAME'] = "timm/eva_large_patch14_336.in22k_ft_in22k_in1k"
#CFG['IMG_SIZE'] = 336
##CFG['BATCH_SIZE'] = 16 ## 16//16G
#CFG['LR'] = [ 0.25e-5 * np.sqrt(CFG['BATCH_SIZE']), 1e-7 ]
# # ----------------------------------
# # [0. --[9836][9842x2]
# [0.9811,0.9825,0.9811,0.9819,0.9837]
# CFG['MODEL_NAME'] = "timm/eva_large_patch14_196.in22k_ft_in22k_in1k"
# CFG['IMG_SIZE'] = 196
# # CFG['IMG_TRAIN_SIZE'] = 196 * 2
# CFG['BATCH_SIZE'] = 48 ## 48/16G(ema), 16/8G
# CFG['LR'] = [ 0.25e-5 * np.sqrt(CFG['BATCH_SIZE']), 1e-6 ]
# # ----------------------------------
# [0.9768, 0.9707, 0.9714, 0.9791, 0.9724 ]
# CFG['MODEL_NAME'] = "timm/convnextv2_large.fcmae_ft_in22k_in1k" ## 288
# # CFG['MODEL_NAME'] = "facebook/convnextv2-large-22k-224"
# CFG['IMG_SIZE'] = 288
# CFG['BATCH_SIZE'] = 16  # 6/8G, 16/16G
# CFG['PRECISION'] = '16'
# CFG['LR'] = [ 0.25e-5 * np.sqrt(CFG['BATCH_SIZE']), 1e-7 ]
# #----------------------------------
# ## best_score=0.9699
# ## A4000: [9737]
# CFG['MODEL_NAME'] = "timm/swinv2_large_window12_192.ms_in22k"
# CFG['IMG_SIZE'] = 192
# CFG['BATCH_SIZE'] = 40 ## 40/16
# CFG['LR'] = 0.25e-5 * np.sqrt(CFG['BATCH_SIZE'])
# # #----------------------------------
# # best_score=0.9805
# [0.9818,0.9815,0.9803,0.9825,0.9813]
# CFG['MODEL_NAME'] = "timm/beitv2_large_patch16_224.in1k_ft_in22k_in1k"
# CFG['IMG_SIZE'] = 224
# CFG['BATCH_SIZE'] = 48 ## 48//16G(ema), 14//8G memory..
# CFG['LR'] = [ 0.25e-5 * np.sqrt(CFG['BATCH_SIZE']), 1e-6 ]
# #----------------------------------
# [0.9742, ]
CFG['MODEL_NAME'] = "timm/deit3_small_patch16_224.fb_in22k_ft_in1k"    ## 304MB
CFG['IMG_SIZE'] = 224
CFG['BATCH_SIZE'] = 8 ## 48//16G, 4//8G memory..
CFG['LR'] = [ 0.25e-5 * np.sqrt(CFG['BATCH_SIZE']), 1e-7 ]
# #----------------------------------

######################################
if 'IMG_TRAIN_SIZE' not in CFG:
    CFG['IMG_TRAIN_SIZE'] = CFG['IMG_SIZE']
logger.info(CFG)

11/20 13:06:44 [INFO] {'SEED': 42, 'N_SPLIT': 3, 'LABEL_SMOOTHING': 0.05, 'OPTIMIZER': 'AdamW', 'INTERPOLATION': 'robidouxsharp', 'PRECISION': '16', 'MODEL_NAME': 'timm/deit3_small_patch16_224.fb_in22k_ft_in1k', 'IMG_SIZE': 224, 'BATCH_SIZE': 8, 'LR': [7.071067811865476e-06, 1e-07], 'IMG_TRAIN_SIZE': 224}


In [ ]:
assert torch.cuda.is_available()
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.set_default_device(device)
logger.info(device)

11/20 13:06:47 [INFO] cuda


In [ ]:
def seed_everything(seed):
    logger.info(f'seed_everything : {seed}')

    import random, os
    import numpy as np
    import torch

    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED'])

11/20 13:06:48 [INFO] seed_everything : 42


In [ ]:
class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, load_img_size, shuffle=False, transforms=None, interpolation='robidouxsharp' ):
        self.df = pd.DataFrame({'img_path_list': img_path_list})
        self.interpolation = interpolation
        self.load_img_size = load_img_size
        logger.info(f'load_img_size={load_img_size}')
        if label_list is not None:
            self.df['label_list'] = label_list
        if shuffle:
            self.df = self.df.sample(frac=1.0).reset_index(drop=True)
        self.transforms = transforms

    # numpy or PIL Image => PIL Image
    def get_interpolated_image(self, img, new_image_size):
        if self.interpolation == 'pil_lanczos':
            if isinstance(img, np.ndarray ):
                img = Image.fromarray(img)
            return img.resize( (new_image_size, new_image_size), Image.LANCZOS )
        elif self.interpolation == 'cv2_lanczos4':
            if not isinstance(img, np.ndarray ):
                img = np.array(img)
            import cv2
            img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
            img = cv2.resize(src, (new_image_size, new_image_size), interpolation=cv2.INTER_LANCZOS4) # 픽셀 크기 지정
            img = cv2.cvtColor(np.array(img), cv2.COLOR_BGR2RGB)
            return Image.fromarray(img)
        else:
            if not isinstance(img, np.ndarray ):
                img = np.array(img)
            from wand import image
            with image.Image.from_array(img) as src:
                src.resize( new_image_size, new_image_size, filter=self.interpolation )
                return Image.fromarray(np.array(src))

    # path => PIL Image
    def get_image_from_index(self, index, img_size ):
        img_path = self.df.img_path_list[index]
        fname = img_path.replace('./','').split('.')[0] + '.png'
        full_fname = f'img_cached/{img_size}_{self.interpolation}/{fname}'
        if os.path.exists(full_fname):
            img = Image.open(full_fname)
        else:
            fname_path = '/'.join(full_fname.split('/')[:-1])
            os.makedirs(fname_path, exist_ok = True)
            img = self.get_interpolated_image(Image.open(img_path), img_size )
            img.save( full_fname )
        return img

    def __getitem__(self, index):
        image = self.get_image_from_index(index, self.load_img_size)
        if self.transforms is not None:
            image = self.transforms(image)
        if 'label_list' in self.df.columns:
            label = torch.tensor(self.df.label_list[index], dtype=torch.long)  # 라벨 타입 변환
            return {'pixel_values': image, 'label': label}
        else:
            return {'pixel_values': image}


    def __len__(self):
        return len(self.df)

In [ ]:
import math
import torch
from torch.optim.lr_scheduler import _LRScheduler

class CosineAnnealingWarmupRestarts(_LRScheduler):
    """
    학습률 스케줄링을 수행하는 클래스.
    주기적으로 학습률을 코사인 형태로 감소시키며, 초기 워밍업 단계를 지원합니다.

    Args:
        optimizer (Optimizer): 학습 중인 옵티마이저.
        first_cycle_steps (int): 첫 번째 주기의 단계 수.
        cycle_mult (float): 다음 주기 단계 수를 증가시키는 배율. 기본값: 1.
        max_lr (float): 첫 번째 주기의 최대 학습률. 기본값: 1e-5.
        min_lr (float): 최소 학습률. 기본값: 1e-10.
        warmup_steps (int): 워밍업 단계 수. 기본값: 0.
        gamma (float): 주기별 최대 학습률 감소율. 기본값: 1.
        last_epoch (int): 마지막 에폭 인덱스. 기본값: -1.
    """

    def __init__(self,
                 optimizer : torch.optim.Optimizer,
                 first_cycle_steps : int,
                 cycle_mult : float = 1.,
                 max_lr : float = 1e-5,
                 min_lr : float = 1e-10,
                 warmup_steps : int = 0,
                 gamma : float = 1.,
                 last_epoch : int = -1):
        assert warmup_steps < first_cycle_steps, "Warmup steps must be less than the total steps in the first cycle."

        # 초기 설정
        self.first_cycle_steps = first_cycle_steps  # 첫 번째 주기 단계 수
        self.cycle_mult = cycle_mult  # 주기 단계 증가 배율
        self.base_max_lr = max_lr  # 초기 최대 학습률
        self.max_lr = max_lr  # 현재 주기의 최대 학습률
        self.min_lr = min_lr  # 최소 학습률
        self.warmup_steps = warmup_steps  # 워밍업 단계 수
        self.gamma = gamma  # 주기별 최대 학습률 감소율

        self.cur_cycle_steps = first_cycle_steps  # 현재 주기 단계 수
        self.cycle = 0  # 주기 횟수
        self.step_in_cycle = last_epoch  # 현재 주기의 단계 인덱스

        # 부모 클래스 초기화
        super(CosineAnnealingWarmupRestarts, self).__init__(optimizer, last_epoch)

        # 학습률 초기화 (최소 학습률로 설정)
        self.init_lr()

    def init_lr(self):
        """최소 학습률로 초기화."""
        self.base_lrs = []
        for param_group in self.optimizer.param_groups:
            param_group['lr'] = self.min_lr
            self.base_lrs.append(self.min_lr)

    def get_lr(self):
        """현재 단계에 따른 학습률 계산."""
        if self.step_in_cycle == -1:
            return self.base_lrs  # 초기 상태의 학습률
        elif self.step_in_cycle < self.warmup_steps:
            # 워밍업 단계: 선형적으로 증가
            return [(self.max_lr - base_lr) * self.step_in_cycle / self.warmup_steps + base_lr for base_lr in self.base_lrs]
        else:
            # 코사인 학습률 감소
            return [base_lr + (self.max_lr - base_lr) *
                    (1 + math.cos(math.pi * (self.step_in_cycle - self.warmup_steps) /
                                  (self.cur_cycle_steps - self.warmup_steps))) / 2
                    for base_lr in self.base_lrs]

    def step(self, epoch=None):
        """
        스케줄러의 현재 상태를 업데이트하고, 학습률을 재계산.

        Args:
            epoch (int, optional): 현재 에폭 번호. 기본값: None.
        """
        if epoch is None:
            # 에폭이 지정되지 않으면 다음 단계로 이동
            epoch = self.last_epoch + 1
            self.step_in_cycle += 1
            if self.step_in_cycle >= self.cur_cycle_steps:
                # 주기가 끝난 경우 다음 주기로 이동
                self.cycle += 1
                self.step_in_cycle -= self.cur_cycle_steps
                self.cur_cycle_steps = int((self.cur_cycle_steps - self.warmup_steps) * self.cycle_mult) + self.warmup_steps
        else:
            # 특정 에폭으로 설정
            if epoch >= self.first_cycle_steps:
                if self.cycle_mult == 1.:
                    self.step_in_cycle = epoch % self.first_cycle_steps
                    self.cycle = epoch // self.first_cycle_steps
                else:
                    n = int(math.log((epoch / self.first_cycle_steps * (self.cycle_mult - 1) + 1), self.cycle_mult))
                    self.cycle = n
                    self.step_in_cycle = epoch - int(self.first_cycle_steps * (self.cycle_mult ** n - 1) / (self.cycle_mult - 1))
                    self.cur_cycle_steps = self.first_cycle_steps * self.cycle_mult ** n
            else:
                self.cur_cycle_steps = self.first_cycle_steps
                self.step_in_cycle = epoch

        # 최대 학습률 감소
        self.max_lr = self.base_max_lr * (self.gamma**self.cycle)
        self.last_epoch = math.floor(epoch)

        # 학습률 업데이트
        for param_group, lr in zip(self.optimizer.param_groups, self.get_lr()):
            param_group['lr'] = lr


In [ ]:
class CustomModel(nn.Module):
    def __init__(self, model):
        super(CustomModel, self).__init__()
        self.model = model
        self.clf = nn.LazyLinear(25)

    def forward(self, x):
        x = self.model(x)
        if not isinstance(x, torch.Tensor):
            x = x.pooler_output
        if self.clf:
            x = self.clf(x)
        return x

In [ ]:
train_df = pd.read_csv('C:/wockd/ultra/ultra/retina/train.csv')
le = LabelEncoder()
train_df['class'] = le.fit_transform(train_df['label'])

In [ ]:
if not len(train_df) == len(os.listdir('C:/wockd/ultra/ultra/retina/train')):
    raise ValueError()

In [ ]:
# 데이터프레임 열 이름 확인
print(train_df.columns)

# 이미지 경로가 저장된 열 이름이 'img_path'라고 가정
missing_files = train_df[~train_df['img_path'].apply(os.path.exists)]
print(f"누락된 파일 수: {len(missing_files)}")
print(missing_files)


Index(['img_path', 'upscale_img_path', 'label', 'class'], dtype='object')
누락된 파일 수: 0
Empty DataFrame
Columns: [img_path, upscale_img_path, label, class]
Index: []


In [ ]:
skf = StratifiedKFold(n_splits=CFG['N_SPLIT'], random_state=CFG['SEED'], shuffle=True)

In [ ]:
image_size = CFG['IMG_SIZE']

train_transform_list = [
    # v2.RandomHorizontalFlip(), ## eva모델등에서는 성능향상이 없음. 오히려 성능떨어짐.
    v2.TrivialAugmentWide(interpolation=v2.InterpolationMode.BICUBIC),
    v2.RandomErasing(),
    v2.Resize(size=(image_size, image_size), interpolation=v2.InterpolationMode.LANCZOS, antialias=True),
    v2.ToImage(), v2.ToDtype( torch.float32, scale=True),
    v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
]
if CFG['IMG_SIZE'] == CFG['IMG_TRAIN_SIZE']:
    train_transform_list = [ a for a in train_transform_list if not isinstance(a, v2.Resize) ]
train_transform = v2.Compose(train_transform_list )
test_transform = v2.Compose( [
    v2.ToImage(), v2.ToDtype( torch.float32, scale=True),
    v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [ ]:
#체크포인트 없는버전 기존코드
from PIL import Image
from sklearn.utils.class_weight import compute_class_weight
class_weight = torch.FloatTensor(compute_class_weight('balanced', classes=train_df.label.sort_values().unique(), y=train_df.label))

def train(model, optimizer, train_loader, val_loader, scheduler, device, validation_steps=0.25, logging_steps=10, use_amp=True):
    logger.info(f'{use_amp=}')

    model.to(device)
    best_score = 0
    best_loss = 1000
    MAX_PATIENCE = 5
    best_patience = MAX_PATIENCE
    loss_fn = nn.CrossEntropyLoss(weight=class_weight, label_smoothing=CFG['LABEL_SMOOTHING'], reduction='mean').to(device)
    scaler = torch.cuda.amp.GradScaler(enabled=use_amp)

    max_steps = len(train_loader)
    if not isinstance(validation_steps, int):
        validation_steps = int(max_steps * validation_steps)  # 절사
    max_steps = (max_steps // validation_steps) * validation_steps

    ema_model = None
    ema_decay = np.power(np.e, np.log(0.5) / (validation_steps * MAX_PATIENCE))
    ema_model = torch.optim.swa_utils.AveragedModel(model, multi_avg_fn=torch.optim.swa_utils.get_ema_multi_avg_fn(ema_decay))

    for epoch in range(1, 50):
        model.train()
        train_loss = []
        pbar_postfix = {}

        pbar = tqdm(train_loader, desc=f'Epoch {epoch}')
        for i, batch in enumerate(pbar):
            if i >= max_steps:
                continue
            steps = i + 1

            if use_amp:
                with torch.autocast(device_type=device, dtype=torch.float16, enabled=use_amp):
                    output = model(batch['pixel_values'])
                    loss = loss_fn(output, batch['label'].to(torch.int64))  # 라벨 타입 변환

                scaler.scale(loss).backward()

                scaler.unscale_(optimizer)
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=0.1)

                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad()
            else:
                output = model(batch['pixel_values'])
                loss = loss_fn(output, batch['label'].to(torch.int64))  # 라벨 타입 변환

                loss.backward()
                optimizer.step()
                optimizer.zero_grad()

            if scheduler is not None:
                scheduler.step()

            train_loss.append(loss.item())
            loss = None
            output = None
            batch = None

            if ema_model is not None:
                ema_model.update_parameters(model)

            if steps % logging_steps == 0:
                pbar_postfix.update({
                    't_loss0': train_loss[-1],
                    'lr': optimizer.param_groups[0]["lr"]
                })
                pbar.set_postfix(pbar_postfix)
                run.log({
                    "epoch": epoch,
                    "step": steps,
                    "train": {"loss": train_loss[-1]},
                    "lr": optimizer.param_groups[0]["lr"]
                }, step=(epoch - 1) * max_steps + steps)

            if steps % validation_steps == 0:
                _val_loss, _val_score = validation(model, loss_fn, val_loader, device, use_amp)
                _train_loss = np.mean(train_loss)

                best_score_mark = '*' if best_score < _val_score else ' '
                best_loss_mark = '*' if best_loss > _val_loss else ' '
                pbar_postfix.update({
                    'lr': optimizer.param_groups[0]["lr"],
                    't_loss': _train_loss,
                    'v_loss': _val_loss,
                    'v_f1': _val_score
                })
                pbar.set_postfix(pbar_postfix)
                logger.info(f'eps={epoch:d}, lr={optimizer.param_groups[0]["lr"]:.3g}, t_loss={_train_loss:.4f}, v_loss={_val_loss:.4f}{best_loss_mark}, v_f1={_val_score:.4f}{best_score_mark}')
                run.log({
                    "epoch": epoch, "step": steps,
                    "train": {"avg_loss": _train_loss},
                    "valid": {"loss": _val_loss, "score": _val_score},
                    "lr": optimizer.param_groups[0]["lr"]
                }, step=(epoch - 1) * max_steps + steps)

                if best_score < _val_score:
                    best_score = _val_score
                    best_patience = MAX_PATIENCE
                    best_loss = min(best_loss, _val_loss)
                elif best_loss > _val_loss:
                    best_loss = _val_loss
                    best_patience = MAX_PATIENCE
                elif best_patience > 0:
                    best_patience -= 1
                else:
                    logger.info(f'NO_MORE_TRAINING, {best_score=:.4f}')
                    if ema_model is not None:
                        torch.optim.swa_utils.update_bn(train_loader, ema_model, device)
                        ema_val_loss, ema_val_score = validation(ema_model, loss_fn, val_loader, device, use_amp)
                        logger.info(f'EMA ::: ema_v_loss={ema_val_loss:.4f}, ema_v_f1={ema_val_score:.4f}')
                        run.log({'ema_v_loss': ema_val_loss, 'ema_v_f1': ema_val_score})
                    return model


In [ ]:
#오수정

def train(model, optimizer, train_loader, val_loader, scheduler, device, validation_steps=0.25, logging_steps=10, use_amp=True):
    logger.info(f'{use_amp=}')
    model.to(device)

    best_score = 0
    best_loss = float('inf')
    loss_fn = nn.CrossEntropyLoss(weight=class_weight, label_smoothing=CFG['LABEL_SMOOTHING'], reduction='mean').to(device)
    scaler = torch.amp.GradScaler(enabled=use_amp)

    max_steps = len(train_loader)
    validation_steps = int(max_steps * validation_steps)

    for epoch in range(1, 15):
        model.train()
        train_loss = []

        pbar = tqdm(train_loader, desc=f'Epoch {epoch}')
        for i, batch in enumerate(pbar):
            inputs = batch['pixel_values'].to(device)
            labels = batch['label'].to(device, dtype=torch.int64)  # 라벨 타입 변환

            optimizer.zero_grad()

            if use_amp:
                with torch.amp.autocast(device_type='cuda', dtype=torch.float16):  # AMP 활성화
                    output = model(inputs)
                    loss = loss_fn(output, labels)
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
            else:
                output = model(inputs)
                loss = loss_fn(output, labels)
                loss.backward()
                optimizer.step()

            train_loss.append(loss.item())
            pbar.set_postfix({'t_loss': train_loss[-1]})

            # Validation 단계 추가
            if (i + 1) % validation_steps == 0:
                val_loss, val_score = validation(model, loss_fn, val_loader, device, use_amp)
                logger.info(f'Epoch {epoch}, Step {i + 1}, Val Loss: {val_loss:.4f}, Val Score: {val_score:.4f}')

    return model


In [ ]:
# AMP 활성화 여부 설정
use_amp = (CFG['PRECISION'] == '16')  # 설정 값이 '16'인 경우 활성화

# GradScaler 초기화
scaler = torch.amp.GradScaler(enabled=use_amp)


In [ ]:
from torch.amp import GradScaler, autocast

scaler = GradScaler(enabled=use_amp)


In [ ]:
#기존코드
def validation(model, loss_fn, val_loader, device, use_amp):
    model = model.to(device)
    save_training = model.training
    model.eval()

    val_loss = []
    preds, true_labels = [], []

    with torch.no_grad():
        for batch in tqdm(val_loader):
            true_labels += batch['label'].detach().cpu().numpy().tolist()
            with torch.autocast(device_type=device, dtype=torch.float16, enabled=use_amp):
                pred = model(batch['pixel_values'])
                loss = loss_fn(output, batch['label'].to(torch.int64))  # 라벨 타입 변환

            preds += pred.detach().argmax(1).cpu().numpy().tolist()
            val_loss.append(loss.item())

        _val_loss = np.mean(val_loss)
        _val_score = f1_score(true_labels, preds, average='macro')
    ## return_to_train..
    if save_training:
        model.train()
    return _val_loss, _val_score

In [ ]:
#오류 수정
def validation(model, loss_fn, val_loader, device, use_amp=True):
    model.eval()
    val_loss = []
    correct = 0
    total = 0

    with torch.no_grad():
        for batch in val_loader:
            inputs = batch['pixel_values'].to(device)
            labels = batch['label'].to(device, dtype=torch.int64)

            if use_amp:
                with torch.amp.autocast():
                    output = model(inputs)
                    loss = loss_fn(output, labels)
            else:
                output = model(inputs)
                loss = loss_fn(output, labels)

            val_loss.append(loss.item())
            _, predicted = torch.max(output, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_val_loss = sum(val_loss) / len(val_loss)
    accuracy = correct / total if total > 0 else 0  # 정확도
    return avg_val_loss, accuracy  # 평균 손실과 정확도를 반환



In [ ]:
def prediction(model, test_loader, device):
    model = model.to(device)
    save_training = model.training
    model.eval()
    preds = []

    with torch.no_grad():
        for batch in tqdm(test_loader):
            pixel_values = batch['pixel_values'].to(device)
            pred = model(pixel_values)  ## F.softmax(output) ## 의미는 없을 듯.
            preds += pred.detach().cpu().numpy().tolist()
    if save_training:
        model.train()
    return preds

In [ ]:
def create_model(model_name):
    import timm
    from transformers import AutoModel, AutoModelForImageClassification, AutoConfig

    logger.info(f'create_model: {model_name}')
    if '/' not in model_name:
        model_name = 'timm/' + model_name

    if model_name.startswith('./'):
        import nextvit
        model = CustomModel(timm.create_model('nextvit_large', pretrained=True))  # checkpoint_path 제거
    elif model_name.startswith('facebook/hiera_'):
        from hiera import Hiera  ## pip install hiera-transformer
        model = CustomModel(Hiera.from_pretrained(model_name))
    elif model_name.startswith('timm/'):
        model = CustomModel(timm.create_model(model_name, pretrained=True))
    else:
        model = CustomModel(AutoModel.from_pretrained(model_name))
    model.eval()
    model(torch.rand((1, 3, CFG['IMG_SIZE'], CFG['IMG_SIZE'])).type(torch.float32))  # initalize_lazyLinear..
    return model


# 모델 훈련

In [ ]:
##체크포인트 제거
from datetime import datetime
dt_str = datetime.now().strftime('%m%d%H%M')

for fold_idx, (train_index, val_index) in enumerate(skf.split(train_df, train_df['class'])):
    gc.collect()
    torch.cuda.empty_cache()

    logger.info(f'{fold_idx=} started')
    import wandb
    run = wandb.init(
        name=f'fold{fold_idx+1}_{CFG["MODEL_NAME"].split("/")[1].split("-")[0]}_{dt_str}',
        config=CFG,
        reinit=True)

    # 학습 및 검증 데이터 분리
    train_fold_df = train_df.loc[train_index, :]
    val_fold_df = train_df.loc[val_index, :]

    # 학습 데이터셋 및 DataLoader 생성
    train_dataset = CustomDataset(
        train_fold_df['img_path'].values, train_fold_df['class'].values,
        interpolation=CFG['INTERPOLATION'], load_img_size=CFG['IMG_TRAIN_SIZE'],
        shuffle=True, transforms=train_transform)
    train_loader = DataLoader(
        train_dataset,
        batch_size=CFG['BATCH_SIZE'],
        shuffle=True,
        generator=torch.Generator(device=device),
        num_workers=4)

    # 검증 데이터셋 및 DataLoader 생성
    val_dataset = CustomDataset(
        val_fold_df['img_path'].values,
        val_fold_df['class'].values,
        interpolation=CFG['INTERPOLATION'], load_img_size=CFG['IMG_SIZE'],
        shuffle=False,
        transforms=test_transform)
    val_loader = DataLoader(
        val_dataset,
        batch_size=CFG['BATCH_SIZE'] * 2,
        shuffle=False,
        generator=torch.Generator(device=device),
        num_workers=4)

    # 모델 생성
    model = create_model(CFG['MODEL_NAME'])

    # 옵티마이저 및 스케줄러 초기화
    optimizer = torch.optim.AdamW(
        model.parameters(),
        lr=CFG['LR'][0],
        weight_decay=0.001)  # 논문에서 추천값은 0.001

    scheduler = CosineAnnealingWarmupRestarts(
        optimizer,
        first_cycle_steps=int(len(train_loader)) // 4,
        cycle_mult=1.0,
        max_lr=CFG['LR'][0] * 2,
        min_lr=CFG['LR'][1],
        warmup_steps=0,
        gamma=0.93)

    # 학습 함수 호출
    model = train(
        model,
        optimizer,
        train_loader,
        val_loader,
        scheduler,
        device,
        use_amp=(CFG['PRECISION'] == '16')  # AMP(Automatic Mixed Precision) 사용 여부
    )

    # 메모리 정리
    model = None
    gc.collect()
    torch.cuda.empty_cache()
    logger.info(f'{fold_idx=} finished')
    run.finish()


11/20 08:26:59 [INFO] fold_idx=0 started


11/20 08:27:00 [INFO] load_img_size=224
11/20 08:27:00 [INFO] load_img_size=224
11/20 08:27:00 [INFO] create_model: timm/deit3_large_patch16_224.fb_in22k_ft_in1k
11/20 08:27:00 [INFO] Loading pretrained weights from Hugging Face hub (timm/deit3_large_patch16_224.fb_in22k_ft_in1k)
11/20 08:27:00 [INFO] [timm/deit3_large_patch16_224.fb_in22k_ft_in1k] Safe alternative available for 'pytorch_model.bin' (as 'model.safetensors'). Loading weights using safetensors.
11/20 08:27:00 [INFO] use_amp=True


C:\Users\wockd\AppData\Local\Temp\ipykernel_5728\2040297209.py:9: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=use_amp)
Epoch 1:   0%|          | 0/2112 [00:00<?, ?it/s]

In [ ]:
#랜덤 1500개
from datetime import datetime
dt_str = datetime.now().strftime('%m%d%H%M')

for fold_idx, (train_index, val_index) in enumerate(skf.split(train_df, train_df['class'])):
    gc.collect()
    torch.cuda.empty_cache()

    logger.info(f'{fold_idx=} started')
    import wandb
    run = wandb.init(
        name=f'fold{fold_idx+1}_{CFG["MODEL_NAME"].split("/")[1].split("-")[0]}_{dt_str}',
        config=CFG,
        reinit=True)

    # 학습 및 검증 데이터 분리
    train_fold_df = train_df.loc[train_index, :]
    val_fold_df = train_df.loc[val_index, :]

    # 학습 데이터 샘플링
    sample_size = min(1500, len(train_fold_df))  # 샘플 크기를 최대 1500으로 제한
    train_fold_df = train_fold_df.sample(n=sample_size, random_state=42).reset_index(drop=True)

    # 학습 데이터셋 및 DataLoader 생성
    train_dataset = CustomDataset(
        train_fold_df['img_path'].values, train_fold_df['class'].values,
        interpolation=CFG['INTERPOLATION'], load_img_size=CFG['IMG_TRAIN_SIZE'],
        shuffle=True, transforms=train_transform)
    train_loader = DataLoader(
        train_dataset,
        batch_size=CFG['BATCH_SIZE'] // 2,
        shuffle=True,
        generator=torch.Generator(device=device),
        num_workers=0)

    # 검증 데이터셋 및 DataLoader 생성
    val_dataset = CustomDataset(
        val_fold_df['img_path'].values,
        val_fold_df['class'].values,
        interpolation=CFG['INTERPOLATION'], load_img_size=CFG['IMG_SIZE'],
        shuffle=False,
        transforms=test_transform)
    val_loader = DataLoader(
        val_dataset,
        batch_size=CFG['BATCH_SIZE'] * 2,
        shuffle=False,
        generator=torch.Generator(device=device),
        num_workers=0)

    # 모델 생성
    model = create_model(CFG['MODEL_NAME'])

    # 옵티마이저 및 스케줄러 초기화
    optimizer = torch.optim.AdamW(
        model.parameters(),
        lr=CFG['LR'][0],
        weight_decay=0.001)  # 논문에서 추천값은 0.001

    scheduler = CosineAnnealingWarmupRestarts(
        optimizer,
        first_cycle_steps=int(len(train_loader)) // 4,
        cycle_mult=1.0,
        max_lr=CFG['LR'][0] * 2,
        min_lr=CFG['LR'][1],
        warmup_steps=0,
        gamma=0.93)

    # 학습 함수 호출
    model = train(
        model,
        optimizer,
        train_loader,
        val_loader,
        scheduler,
        device,
        use_amp=(CFG['PRECISION'] == '16')  # AMP(Automatic Mixed Precision) 사용 여부
    )

    # 메모리 정리
    model = None
    gc.collect()
    torch.cuda.empty_cache()
    logger.info(f'{fold_idx=} finished')
    run.finish()


NameError: name 'skf' is not defined

In [ ]:
# 이미지 랜덤 1500 및 오류 수정 / 폴드5에 에폭 50 T_loss 1이상
from datetime import datetime
dt_str = datetime.now().strftime('%m%d%H%M')

# K-Fold Cross-Validation 학습 루프
for fold_idx, (train_index, val_index) in enumerate(skf.split(train_df, train_df['class'])):
    gc.collect()
    torch.cuda.empty_cache()

    logger.info(f'{fold_idx=} started')

    # W&B 초기화
    import wandb
    run = wandb.init(
        name=f'fold{fold_idx+1}_{CFG["MODEL_NAME"].split("/")[1].split("-")[0]}_{dt_str}',
        config=CFG,
        reinit=True)

    # 학습 및 검증 데이터 분리
    train_fold_df = train_df.loc[train_index, :]
    val_fold_df = train_df.loc[val_index, :]

    # 학습 데이터 샘플링
    sample_size = min(1500, len(train_fold_df))  # 샘플 크기를 최대 1500으로 제한
    train_fold_df = train_fold_df.sample(n=sample_size, random_state=42).reset_index(drop=True)

    # 학습 데이터셋 및 DataLoader 생성
    train_dataset = CustomDataset(
        train_fold_df['img_path'].values, train_fold_df['class'].values,
        interpolation=CFG['INTERPOLATION'], load_img_size=CFG['IMG_TRAIN_SIZE'],
        shuffle=True, transforms=train_transform)
    train_loader = DataLoader(
        train_dataset,
        batch_size=CFG['BATCH_SIZE'] // 2,
        shuffle=True,
        generator=torch.Generator(device=device),
        num_workers=0)

    # 검증 데이터셋 및 DataLoader 생성
    val_dataset = CustomDataset(
        val_fold_df['img_path'].values,
        val_fold_df['class'].values,
        interpolation=CFG['INTERPOLATION'], load_img_size=CFG['IMG_SIZE'],
        shuffle=False,
        transforms=test_transform)
    val_loader = DataLoader(
        val_dataset,
        batch_size=CFG['BATCH_SIZE'] * 2,
        shuffle=False,
        generator=torch.Generator(device=device),
        num_workers=0)

    # 모델 생성
    model = create_model(CFG['MODEL_NAME']).to(device)

    # 옵티마이저 및 스케줄러 초기화
    optimizer = torch.optim.AdamW(
        model.parameters(),
        lr=CFG['LR'][0],
        weight_decay=0.001)  # 논문에서 추천값은 0.001

    scheduler = CosineAnnealingWarmupRestarts(
        optimizer,
        first_cycle_steps=int(len(train_loader)) // 4,
        cycle_mult=1.0,
        max_lr=CFG['LR'][0] * 2,
        min_lr=CFG['LR'][1],
        warmup_steps=0,
        gamma=0.93)

    # 학습 함수 호출 (train 함수 사용)
    model = train(
        model=model,
        optimizer=optimizer,
        train_loader=train_loader,
        val_loader=val_loader,
        scheduler=scheduler,
        device=device,
        use_amp=(CFG['PRECISION'] == '16')  # AMP 사용 여부
    )

    # 메모리 정리
    model = None
    gc.collect()
    torch.cuda.empty_cache()
    logger.info(f'{fold_idx=} finished')
    run.finish()

#이게 성공한 코드

11/20 08:47:36 [INFO] fold_idx=0 started


11/20 08:47:38 [INFO] load_img_size=224
11/20 08:47:38 [INFO] load_img_size=224
11/20 08:47:38 [INFO] create_model: timm/deit3_large_patch16_224.fb_in22k_ft_in1k
11/20 08:47:38 [INFO] Loading pretrained weights from Hugging Face hub (timm/deit3_large_patch16_224.fb_in22k_ft_in1k)
11/20 08:47:38 [INFO] [timm/deit3_large_patch16_224.fb_in22k_ft_in1k] Safe alternative available for 'pytorch_model.bin' (as 'model.safetensors'). Loading weights using safetensors.
11/20 08:47:39 [INFO] use_amp=True
11/20 08:47:39 [ERRO] -------Exception----------
Traceback (most recent call last):
  File "c:\Users\wockd\anaconda3\envs\newwoc\lib\site-packages\IPython\core\interactiveshell.py", line 3550, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\wockd\AppData\Local\Temp\ipykernel_25380\2282410872.py", line 72, in <module>
    model = train(
  File "C:\Users\wockd\AppData\Local\Temp\ipykernel_25380\4110448684.py", line 8, in train
    loss_fn = nn.CrossEntropyLoss(weig

In [ ]:
#최종?
from datetime import datetime
from sklearn.utils.class_weight import compute_class_weight
import gc
import wandb
from torch.utils.data import DataLoader

# 현재 시간 문자열 생성
dt_str = datetime.now().strftime('%m%d%H%M')

# K-Fold Cross-Validation 학습 루프
best_score = 0.0  # 최적의 검증 점수 초기화
best_model = None  # 최적의 모델 초기화

for fold_idx, (train_index, val_index) in enumerate(skf.split(train_df, train_df['class'])):
    gc.collect()
    torch.cuda.empty_cache()

    logger.info(f'{fold_idx=} started')

    # W&B 초기화
    run = wandb.init(
        name=f'fold{fold_idx+1}_{CFG["MODEL_NAME"].split("/")[1].split("-")[0]}_{dt_str}',
        config=CFG,
        reinit=True)

    # 학습 및 검증 데이터 분리
    train_fold_df = train_df.loc[train_index, :]
    val_fold_df = train_df.loc[val_index, :]

    # 학습 데이터 샘플링
    sample_size = min(1500, len(train_fold_df))  # 샘플 크기를 최대 1500으로 제한
    train_fold_df = train_fold_df.sample(n=sample_size, random_state=42).reset_index(drop=True)

    # 클래스 가중치 계산
    class_weight = torch.FloatTensor(compute_class_weight(
        'balanced',
        classes=train_fold_df['class'].unique(),
        y=train_fold_df['class']
    )).to(device)

    # 학습 데이터셋 및 DataLoader 생성
    train_dataset = CustomDataset(
        train_fold_df['img_path'].values, train_fold_df['class'].values,
        interpolation=CFG['INTERPOLATION'], load_img_size=CFG['IMG_TRAIN_SIZE'],
        shuffle=True, transforms=train_transform)
    train_loader = DataLoader(
        train_dataset,
        batch_size=CFG['BATCH_SIZE'] // 2,
        shuffle=True,
        generator=torch.Generator(device=device),
        num_workers=0)

    # 검증 데이터셋 및 DataLoader 생성
    val_dataset = CustomDataset(
        val_fold_df['img_path'].values,
        val_fold_df['class'].values,
        interpolation=CFG['INTERPOLATION'], load_img_size=CFG['IMG_SIZE'],
        shuffle=False,
        transforms=test_transform)
    val_loader = DataLoader(
        val_dataset,
        batch_size=CFG['BATCH_SIZE'] * 2,
        shuffle=False,
        generator=torch.Generator(device=device),
        num_workers=0)

    # 모델 생성
    model = create_model(CFG['MODEL_NAME']).to(device)

    # 옵티마이저 및 스케줄러 초기화
    optimizer = torch.optim.AdamW(
        model.parameters(),
        lr=CFG['LR'][0],
        weight_decay=0.001  # 논문에서 추천값은 0.001
    )

    scheduler = CosineAnnealingWarmupRestarts(
        optimizer,
        first_cycle_steps=int(len(train_loader)) // 4,
        cycle_mult=1.0,
        max_lr=CFG['LR'][0] * 2,
        min_lr=CFG['LR'][1],
        warmup_steps=0,
        gamma=0.93
    )

    # 학습 함수 호출
    model, val_score = train(
        model=model,
        optimizer=optimizer,
        train_loader=train_loader,
        val_loader=val_loader,
        scheduler=scheduler,
        device=device,
        class_weight=class_weight,  # 클래스 가중치 전달
        use_amp=(CFG['PRECISION'] == '16')  # AMP 사용 여부
    )

    # 최적의 모델 저장
    if val_score > best_score:
        best_score = val_score
        best_model = model
        torch.save(model.state_dict(), f"best_model_fold_{fold_idx}.pth")
        print(f"최적의 모델 저장 (Fold {fold_idx}, val_score={val_score:.4f})")

    # 메모리 정리
    model = None
    gc.collect()
    torch.cuda.empty_cache()
    logger.info(f'{fold_idx=} finished')
    run.finish()

# 가장 성능이 좋은 Fold 저장
if best_model is not None:
    torch.save(best_model.state_dict(), "final_best_model.pth")
    print("전체 Fold 중 최적의 모델이 저장되었습니다.")
else:
    print("모델이 저장되지 않았습니다.")


11/20 12:05:06 [INFO] fold_idx=0 started


11/20 12:05:07 [INFO] load_img_size=224
11/20 12:05:07 [INFO] load_img_size=224
11/20 12:05:07 [INFO] create_model: timm/deit3_small_patch16_224.fb_in22k_ft_in1k
11/20 12:05:07 [INFO] Loading pretrained weights from Hugging Face hub (timm/deit3_small_patch16_224.fb_in22k_ft_in1k)
11/20 12:05:08 [INFO] [timm/deit3_small_patch16_224.fb_in22k_ft_in1k] Safe alternative available for 'pytorch_model.bin' (as 'model.safetensors'). Loading weights using safetensors.


C:\Users\wockd\AppData\Local\Temp\ipykernel_25380\1940753084.py:4: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=use_amp)
C:\Users\wockd\AppData\Local\Temp\ipykernel_25380\1940753084.py:21: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(dtype=torch.float16):
c:\Users\wockd\anaconda3\envs\newwoc\lib\site-packages\torch\autograd\graph.py:825: UserWarning: cuDNN SDPA backward got grad_output.strides() != output.strides(), attempting to materialize a grad_output with matching strides... (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\cudnn\MHA.cpp:676.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


Epoch 1, Train Loss: 3.4121, Validation Accuracy: 0.0411
Epoch 2, Train Loss: 2.2405, Validation Accuracy: 0.8162
Epoch 3, Train Loss: 0.8413, Validation Accuracy: 0.8890
Epoch 4, Train Loss: 0.5732, Validation Accuracy: 0.9017
Epoch 5, Train Loss: 0.4532, Validation Accuracy: 0.9000
Epoch 6, Train Loss: 0.4075, Validation Accuracy: 0.9070
Epoch 7, Train Loss: 0.3296, Validation Accuracy: 0.9028
Epoch 8, Train Loss: 0.3081, Validation Accuracy: 0.9022
Epoch 9, Train Loss: 0.2854, Validation Accuracy: 0.9153
Epoch 10, Train Loss: 0.2694, Validation Accuracy: 0.9104
Epoch 11, Train Loss: 0.2732, Validation Accuracy: 0.9130
Epoch 12, Train Loss: 0.2786, Validation Accuracy: 0.9138
Epoch 13, Train Loss: 0.2215, Validation Accuracy: 0.9136
Epoch 14, Train Loss: 0.2268, Validation Accuracy: 0.9128
최적의 모델 저장 (Fold 0, val_score=0.9153)
11/20 12:14:21 [INFO] fold_idx=0 finished


11/20 12:14:22 [INFO] fold_idx=1 started


...

11/20 12:25:28 [INFO] fold_idx=2 started


11/20 12:25:28 [INFO] load_img_size=224
11/20 12:25:28 [INFO] load_img_size=224
11/20 12:25:28 [INFO] create_model: timm/deit3_small_patch16_224.fb_in22k_ft_in1k
11/20 12:25:28 [INFO] Loading pretrained weights from Hugging Face hub (timm/deit3_small_patch16_224.fb_in22k_ft_in1k)
11/20 12:25:28 [INFO] [timm/deit3_small_patch16_224.fb_in22k_ft_in1k] Safe alternative available for 'pytorch_model.bin' (as 'model.safetensors'). Loading weights using safetensors.


C:\Users\wockd\AppData\Local\Temp\ipykernel_25380\1940753084.py:4: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=use_amp)
C:\Users\wockd\AppData\Local\Temp\ipykernel_25380\1940753084.py:21: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(dtype=torch.float16):
c:\Users\wockd\anaconda3\envs\newwoc\lib\site-packages\torch\autograd\graph.py:825: UserWarning: cuDNN SDPA backward got grad_output.strides() != output.strides(), attempting to materialize a grad_output with matching strides... (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\cudnn\MHA.cpp:676.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


Epoch 1, Train Loss: 3.3305, Validation Accuracy: 0.0388
Epoch 2, Train Loss: 2.0940, Validation Accuracy: 0.8236
Epoch 3, Train Loss: 0.7321, Validation Accuracy: 0.8772
Epoch 4, Train Loss: 0.5322, Validation Accuracy: 0.8920
Epoch 5, Train Loss: 0.4047, Validation Accuracy: 0.9047
Epoch 6, Train Loss: 0.4098, Validation Accuracy: 0.9072
Epoch 7, Train Loss: 0.3829, Validation Accuracy: 0.9034
Epoch 8, Train Loss: 0.3310, Validation Accuracy: 0.9058
Epoch 9, Train Loss: 0.2728, Validation Accuracy: 0.9047
Epoch 10, Train Loss: 0.2467, Validation Accuracy: 0.9149
Epoch 11, Train Loss: 0.3120, Validation Accuracy: 0.9064
Epoch 12, Train Loss: 0.2435, Validation Accuracy: 0.9077
Epoch 13, Train Loss: 0.2041, Validation Accuracy: 0.9130
Epoch 14, Train Loss: 0.2038, Validation Accuracy: 0.9102
11/20 12:36:10 [INFO] fold_idx=2 finished


전체 Fold 중 최적의 모델이 저장되었습니다.


# 모델 앙상블 및 추론

In [ ]:
test_df = pd.read_csv('C:/wockd/ultra/ultra/retina/test.csv')

In [ ]:
import os

ckpt_dir = 'C:\\wockd\\ultra\\ultra\\retina\\ckpt_kfold'
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)
    print(f"Directory '{ckpt_dir}' created.")


In [ ]:
#체크포인트 - 사용 xxxx

# 체크포인트 파일 리스트 생성
ckpt_df = pd.DataFrame({'fname': glob('./ckpt/*.ckpt')})

# 문자열 변환 (NaN 처리 포함)
ckpt_df['fname'] = ckpt_df['fname'].fillna('').astype(str)

# 파일 수정 시간 추가
ckpt_df['mtime'] = ckpt_df['fname'].apply(lambda x: int(os.stat(x).st_mtime) if x else 0)

# 모델 이름 추출
ckpt_df['model_name'] = ckpt_df['fname'].apply(
    lambda x: re.search(r'./ckpt/(.*?)-fold', x)[1] if re.search(r'./ckpt/(.*?)-fold', x) else 'unknown')

# 이미지 사이즈 추출 (없으면 기본값 0)
ckpt_df['img_size'] = ckpt_df['fname'].apply(
    lambda x: int(re.search(r'patch[0-9]+_([0-9]+)', x + 'patch0_0')[1]) if re.search(r'patch[0-9]+_([0-9]+)', x) else 0)

# EMA 체크
ckpt_df['is_ema'] = ckpt_df['fname'].str.endswith('ema.ckpt').astype(int)

# fold index 추출
ckpt_df['fold_idx'] = ckpt_df['fname'].apply(
    lambda x: int(re.search(r'fold_idx=([0-9])-', x)[1]) if re.search(r'fold_idx=([0-9])-', x) else -1)

# validation loss 추출 (없으면 기본값 0.0)
ckpt_df['val_loss'] = ckpt_df['fname'].apply(
    lambda x: float(re.search(r'val_loss=(0\.[0-9]+)', x)[1]) if re.search(r'val_loss=(0\.[0-9]+)', x) else 0.0)

# validation score 추출 (없으면 기본값 0.0)
ckpt_df['val_score'] = ckpt_df['fname'].apply(
    lambda x: float(re.search(r'val_score=(0\.[0-9]+)', x)[1]) if re.search(r'val_score=(0\.[0-9]+)', x) else 0.0)


In [ ]:
#폴드 3개 앙상블 오류 수정
import pandas as pd
import numpy as np
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm


# 모델 생성 함수
def create_model(model_name):
    import timm
    model = timm.create_model(model_name, pretrained=False)
    model.eval()
    return model

# 예측 함수
def predict(model, data_loader, device):
    preds = []
    with torch.no_grad():
        for images in tqdm(data_loader, desc="Predicting"):
            images = images.to(device)
            outputs = model(images)
            preds.append(torch.softmax(outputs, dim=1).cpu().numpy())
    return np.concatenate(preds, axis=0)

# 앙상블 예측 함수
def ensemble_predictions(model_paths, model_name, test_loader, device):
    ensemble_preds = []
    for model_path in model_paths:
        # 모델 로드
        model = create_model(model_name).to(device)
        state_dict = torch.load(model_path, map_location=device)
        model.load_state_dict(state_dict, strict=False)  # strict=False로 불일치 허용

        # 예측 수행
        preds = predict(model, test_loader, device)
        ensemble_preds.append(preds)

    # 모든 모델의 예측값 평균
    final_preds = np.mean(ensemble_preds, axis=0)
    return final_preds

# 설정
model_paths = [
    r"C:\wockd\ultra\ultra\retina\ckpt_kfold\best_model_fold_0.pth",
    r"C:\wockd\ultra\ultra\retina\ckpt_kfold\best_model_fold_1.pth",
    r"C:\wockd\ultra\ultra\retina\ckpt_kfold\final_best_model.pth"
]
model_name = "deit3_small_patch16_224.fb_in22k_ft_in1k"  # 학습에 사용한 모델 이름
test_csv_path = r"C:\wockd\ultra\ultra\retina\test.csv"
output_file = r"C:\wockd\ultra\ultra\retina\final_predictions.csv"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 테스트 데이터 로드
test_df = pd.read_csv(test_csv_path)

# 데이터셋 생성
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, img_paths, labels=None, transforms=None):
        self.img_paths = img_paths
        self.labels = labels  # labels가 필요하다면 추가
        self.transforms = transforms

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        from PIL import Image
        img = Image.open(self.img_paths[idx]).convert("RGB")
        if self.transforms:
            img = self.transforms(img)
        return img

# 이미지 변환 설정 (필요에 따라 수정)
from torchvision import transforms
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# 테스트 데이터셋 및 DataLoader
test_dataset = CustomDataset(
    img_paths=test_df['img_path'].values,
    transforms=test_transform
)

test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=0)

# 앙상블 수행
final_predictions = ensemble_predictions(model_paths, model_name, test_loader, device)

# 최종 클래스 결정
final_labels = np.argmax(final_predictions, axis=1)

# 결과 저장
test_df['label'] = final_labels
test_df[['id', 'label']].to_csv(output_file, index=False)
print(f"앙상블 결과가 {output_file}에 저장되었습니다.")


C:\Users\wockd\AppData\Local\Temp\ipykernel_2368\2764215821.py:31: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(model_path, map_location=device)
Pre

앙상블 결과가 C:\wockd\ultra\ultra\retina\final_predictions.csv에 저장되었습니다.


In [ ]:
#또 오류 수정,,,
import pandas as pd
import numpy as np
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm
from torchvision import transforms
from PIL import Image

# Model creation function
def create_model(model_name):
    import timm
    model = timm.create_model(model_name, pretrained=False)
    model.eval()
    return model

# Prediction function
def predict(model, data_loader, device):
    preds = []
    with torch.no_grad():
        for images in tqdm(data_loader, desc="Predicting"):
            images = images.to(device)
            outputs = model(images)
            preds.append(torch.softmax(outputs, dim=1).cpu().numpy())
    return np.concatenate(preds, axis=0)

# Ensemble prediction function
def ensemble_predictions(model_paths, model_name, test_loader, device):
    ensemble_preds = []
    for model_path in model_paths:
        model = create_model(model_name).to(device)
        state_dict = torch.load(model_path, map_location=device)
        model.load_state_dict(state_dict, strict=False)

        preds = predict(model, test_loader, device)
        ensemble_preds.append(preds)

    final_preds = np.mean(ensemble_preds, axis=0)
    return final_preds

# Settings
model_paths = [
    r"C:\wockd\ultra\ultra\retina\ckpt_kfold\best_model_fold_0.pth",
    r"C:\wockd\ultra\ultra\retina\ckpt_kfold\best_model_fold_1.pth",
    r"C:\wockd\ultra\ultra\retina\ckpt_kfold\final_best_model.pth"
]
model_name = "deit3_small_patch16_224.fb_in22k_ft_in1k"
test_csv_path = r"C:\wockd\ultra\ultra\retina\test.csv"
train_csv_path = r"C:\wockd\ultra\ultra\retina\train.csv"
output_file = r"C:\wockd\ultra\ultra\retina\kia2.csv"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Label mapping
train_df = pd.read_csv(train_csv_path, header=None, names=['img_path', 'upscale_img_path', 'class_name'])
label_mapping = {i: class_name for i, class_name in enumerate(train_df['class_name'].unique())}

# Check if missing labels exist
print("Label mapping keys:", label_mapping.keys())

# Test data loading
test_df = pd.read_csv(test_csv_path)

# Dataset definition
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, img_paths, transforms=None):
        self.img_paths = img_paths
        self.transforms = transforms

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        img = Image.open(self.img_paths[idx]).convert("RGB")
        if self.transforms:
            img = self.transforms(img)
        return img

# Image transformations
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Test dataset and DataLoader
test_dataset = CustomDataset(
    img_paths=test_df['img_path'].values,
    transforms=test_transform
)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=0)

# Perform ensemble
final_predictions = ensemble_predictions(model_paths, model_name, test_loader, device)

# Map predicted labels to class names with default handling
final_labels = np.argmax(final_predictions, axis=1)
final_class_names = [label_mapping.get(label, "Unknown Class") for label in final_labels]

missing_labels = [label for label in final_labels if label not in label_mapping]
if missing_labels:
    print(f"Warning: Missing mappings for labels {missing_labels}. Check train.csv.")

# Save results
test_df['label'] = final_class_names
test_df[['id', 'label']].to_csv(output_file, index=False)
print(f"Final predictions saved to {output_file}.")


C:\Users\wockd\AppData\Local\Temp\ipykernel_2368\2719723625.py:31: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(model_path, map_location=device)


Label mapping keys: dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25])


Predicting: 100%|██████████| 107/107 [00:14<00:00,  7.26it/s]


Final predictions saved to C:\wockd\ultra\ultra\retina\kia2.csv.


In [ ]:
# 각 K-Fold에서 생성된 best.pth 파일 경로 리스트
model_paths = [
    r"C:\\wockd\\ultra\\ultra\\retina\\ckpt_kfold\\best_model_fold_0.pth",
    r"C:\\wockd\\ultra\\ultra\\retina\\ckpt_kfold\\best_model_fold_1.pth",
    r"C:\\wockd\\ultra\\ultra\\retina\\ckpt_kfold\\final_best_model.pth"
]


In [ ]:
import re

ckpt_df = pd.DataFrame({'fname':glob('./ckpt/*.ckpt')})
ckpt_df['mtime'] = ckpt_df.fname.apply(lambda x: int(os.stat(x).st_mtime))
ckpt_df['model_name'] = ckpt_df.fname.apply(lambda x: re.search(r'./ckpt/(.*?)-fold',x)[1])
ckpt_df['img_size'] = ckpt_df.fname.apply(lambda x: int(re.search(r'patch[0-9]+_([0-9]+)', x + 'patch0_0')[1]) )
ckpt_df['is_ema'] = ckpt_df.fname.str.endswith('ema.ckpt').astype(int)
ckpt_df['fold_idx'] = ckpt_df.fname.apply(lambda x: int(re.search(r'fold_idx=([0-9])-',x)[1]))
ckpt_df['val_loss'] = ckpt_df.fname.apply(lambda x: float(re.search(r'val_loss=(0\.[0-9]+)', x)[1]) )
ckpt_df['val_score'] = ckpt_df.fname.apply(lambda x: float(re.search(r'val_score=(0\.[0-9]+)', x)[1]) )

In [ ]:
ckpt_df = ckpt_df[ckpt_df.img_size != 0][ckpt_df.is_ema == 0]
ckpt_df = ckpt_df.sort_values('mtime',ascending=False).reset_index(drop=True)
ckpt_indexes = ckpt_df[ ckpt_df.fold_idx==ckpt_df.fold_idx.max() ].index[:4]

In [ ]:
print(ckpt_df)
print(ckpt_indexes )

In [ ]:
print("CFG['IMG_SIZE']:", CFG['IMG_SIZE'])
assert CFG['IMG_SIZE'] in (192, 224, 336,)


In [ ]:
#from PIL import Image

preds = []
preds_score = []

for ckpt_start_index in ckpt_indexes:
    logger.info(f'{ckpt_df.fname[ckpt_start_index]} loading')
    ## imagesize
    CFG['IMG_SIZE'] = ckpt_df.img_size[ckpt_start_index]
    assert CFG['IMG_SIZE'] in ( 196, 224, )
    logger.info(CFG['IMG_SIZE'])

    test_dataset = CustomDataset(
        test_df['img_path'].values, None,
        interpolation=CFG['INTERPOLATION'], load_img_size=CFG['IMG_SIZE'],
        shuffle=False, transforms=test_transform)
    test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE']*2, shuffle=False, num_workers=0)

    model_name = ckpt_df.model_name[ckpt_start_index]
    model = create_model(model_name)
    if ckpt_df.is_ema[ckpt_start_index]:
        model = torch.optim.swa_utils.AveragedModel(model)
    #-----------------------------
    for i in range(ckpt_start_index, ckpt_start_index + ckpt_df.fold_idx.max() + 1 ):
        checkpoint_path = ckpt_df.fname[i]
        logger.info(f'{checkpoint_path} loading')
        model.load_state_dict( torch.load(checkpoint_path)['model'] )

        preds_score.append( ckpt_df.val_score[i] )
        preds.append( prediction(model, test_loader, device) )

preds = np.array(preds)
preds_score = np.array(preds_score)

11/19 08:22:35 [INFO] ./ckpt/deit3_large_patch16_224.fb_in22k_ft_in1k-fold_idx=0-epoch=08-val_loss=0.4284-val_score=0.9783.ckpt loading
11/19 08:22:35 [INFO] 224
11/19 08:22:35 [INFO] load_img_size=224
11/19 08:22:35 [INFO] create_model: deit3_large_patch16_224.fb_in22k_ft_in1k
11/19 08:22:35 [INFO] Loading pretrained weights from Hugging Face hub (timm/deit3_large_patch16_224.fb_in22k_ft_in1k)
11/19 08:22:35 [INFO] [timm/deit3_large_patch16_224.fb_in22k_ft_in1k] Safe alternative available for 'pytorch_model.bin' (as 'model.safetensors'). Loading weights using safetensors.
11/19 08:22:40 [INFO] ./ckpt/deit3_large_patch16_224.fb_in22k_ft_in1k-fold_idx=0-epoch=08-val_loss=0.4284-val_score=0.9783.ckpt loading
<ipython-input-28-132539c7501f>:27: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling 

  0%|          | 0/71 [00:00<?, ?it/s]

11/19 08:41:34 [ERRO] -------Exception----------
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-28-132539c7501f>", line 30, in <cell line: 6>
    preds.append( prediction(model, test_loader, device) )
  File "<ipython-input-20-5c04b002d755>", line 8, in prediction
    for batch in tqdm(test_loader):
  File "/usr/local/lib/python3.10/dist-packages/tqdm/notebook.py", line 250, in __iter__
    for obj in it:
  File "/usr/local/lib/python3.10/dist-packages/tqdm/std.py", line 1181, in __iter__
    for obj in iterable:
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 701, in __next__
    data = self._next_data()
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 757, in _next_data
    data = self._dataset_fetcher.fetch(index)  # may raise StopIteration


In [ ]:
# ### 가중치 평균값..
preds_error = (1-preds_score)  ## L1 ACC 오차인경우
preds_error = 1-preds_error/preds_error.sum()
preds_coef = preds_error/preds_error.sum()

logger.info(f'{preds_score=}')
logger.info(f'{preds_coef=}')
preds2 = np.array( [ coef * preds[i] for i, coef in enumerate( preds_coef ) ] )
preds_labels = le.inverse_transform(preds2.sum(0).argmax(-1))
print(preds_labels)

In [ ]:
submit = pd.read_csv('./sample_submission.csv')
submit['label'] = preds_labels
from datetime import datetime
dt_str = datetime.now().strftime('%Y%m%d_%H%M')
submit.to_csv(f'./basslibrary_submit_{dt_str}.csv', index=False)
logger.info(f'./basslibrary_submit_{dt_str}.csv saved')

In [ ]:
submit.label.value_counts()

In [ ]:
# !python ~/send_telegram.py 'basslibrary_submit_{dt_str}.csv saved'